Using this Towards Data Science [article](https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22)

This is a better [example](https://colab.research.google.com/drive/1hMLd5-r82FrnFnBub-B-fVW78Px4KPX1#scrollTo=sm3lGfQb-1J8).  The above was hacked together from this one and is missing the vocab file for tokenization.

This uses TensorFlow 2.0+ and TensorFlow Hub 0.7+

In [1]:
import tensorflow_hub as hub
import tensorflow as tf
import bert
#from bert import tokenization
FullTokenizer = bert.bert_tokenization.FullTokenizer
from tensorflow.keras.models import Model
# Keras is the new high level API for Tensorflow
import math
import pandas as pd

In [2]:
#!pip install bert-for-tf2
#!pip install sentencepiece

In [2]:
max_seq_length = 128 # your choice here

#is_predicting = False
num_labels = 2

# Preprocessing
The BERT layer requires 3 input sequence:

- Token ids: for every token in the sentence.  We restore it from the BERT vocab dictionary
- Mask ids: for every toekn to mask out tokens used only the sequence padding (so every sequence ahs the same length).
- Segment ids: 0 for one-sentence sequence, 1 if there are two sentences in teh sequence and it is the second one (see the original paper or the corresponding part o the BERT on GitHub for more details: `convert_single_example` in the [`run_classifier.py`](https://github.com/google-research/bert/blob/master/run_classifier.py)

In [3]:
def get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than the max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length - len(token_ids))
    return input_ids

# Download data

In [4]:
from tensorflow import keras
import os
import re

# Load all files from the director in a DataFrame
def load_directory_data(directory):
    data = {}
    data["sentence"] = []
    data["sentiment"] = []
    for file_path in os.listdir(directory):
        with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
            data["sentence"].append(f.read())
            data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
    return pd.DataFrame.from_dict(data)

# merge positive and negative examples, add a polarity column and shuffle
def load_dataset(directory):
    pos_df = load_directory_data(os.path.join(directory, "pos"))
    neg_df = load_directory_data(os.path.join(directory, "neg"))
    pos_df["polarity"] = 1
    neg_df["polarity"] = 0
    return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files
def download_and_load_datasets(force_download=False):
    dataset = tf.keras.utils.get_file(
    fname = 'aclImdb.tar.gz',
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz",
    extract=True)
    
    train_df = load_dataset(os.path.join(os.path.dirname(dataset),
                                        "aclImdb", "train"))
    test_df = load_dataset(os.path.join(os.path.dirname(dataset),
                                       "aclImdb", "test"))
    
    return train_df, test_df

In [5]:
download = False
if download:
    train, test = download_and_load_datasets()
    train.to_csv('\saved_data\train.csv')
    test.to_csv('\saved_data\test.csv')
else:
    train = pd.read_csv('\saved_data\train.csv')
    test = pd.read_csv('\saved_data\test.csv')

In [6]:
train = train.sample(1024)
test = test.sample(1024)

In [7]:
x_train, y_train = train['sentence'],train['polarity'].values
x_test, y_test = test['sentence'],test['polarity'].values


# Model Creation
The goal of this model is to use the pre-trained BERT to gernate teh embeddoing vectors.  Tehrefore, we need only the required inputfs for the BERT layer and themodel has only the BERT layer as a hidden layer.  Of course, inside BERT, there is more cpomplex architecture.

The `hub.KerasLayer` function oimports the pre-trained model as a keras layer.

In [8]:
from tensorflow.keras import layers

In [10]:
class Linear(layers.Layer):
    def __init__(self, num_labels=32, hidden_size=32):
        super(Linear, self).__init__()
        self.output_weights = tf.Variable(
            initial_value=tf.random.truncated_normal([num_labels, hidden_size],stddev=0.02),
            dtype='float32', trainable=True)
        self.bias_weights = tf.Variable(tf.zeros([num_labels]),
                       dtype='float32', trainable=True)
        
        
    def call(self, inputs):
        output_layer = tf.nn.dropout(inputs, rate=0.1)

        logits = tf.matmul(output_layer, self.output_weights, transpose_b=True)
        logits = tf.nn.bias_add(logits, self.bias_weights)
        log_probs = tf.nn.log_softmax(logits, axis=-1)

        return log_probs
    
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'output_weights': self.output_weights,
            'bias_weights': self.bias_weights,
            
        })
        return config

In [11]:
input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                      name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="segment_ids")
get_bert = False
if get_bert:
    bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
        trainable=False)
    bert_model = Model(inputs=[input_word_ids, input_mask, segment_ids],
              outputs=[pooled_output, sequence_output])
    bert_model.save('bert_model.h5')
else:
    bert_model = tf.keras.models.load_model('bert_model.h5')
    l1, l2, l3, bert_layer = bert_model.layers
                            
                            
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

output_layer = pooled_output #we're only using the pooled output for classification

hidden_size = output_layer.shape[-1]

In [17]:
"""
# create our own layer to tune
output_weights = tf.Variable(
  tf.random.truncated_normal([num_labels, hidden_size],stddev=0.02),
  name="output_weights")

output_bias = tf.Variable(tf.zeros([num_labels]),
                       name="output_bias")

    #Dropout helps prevent overfitting
output_layer = tf.nn.dropout(output_layer, rate=0.1)

logits = tf.matmul(output_layer, output_weights, transpose_b=True)
logits = tf.nn.bias_add(logits, output_bias)
log_probs = tf.nn.log_softmax(logits, axis=-1)
"""
linear_layer = Linear(num_labels,hidden_size)
log_probs = linear_layer(output_layer)

#log_probs = layers.Dense(2, activation='softmax')(output_layer)

#predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
# If we're predicting, we want predicted labels and the probabiltiies.
#if is_predicting:
#    return (predicted_labels, log_probs)

# If we're train/eval, compute loss between predicted and actual label

#per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
#loss = tf.reduce_mean(per_example_loss)
    
    
model = Model(inputs=[input_word_ids, input_mask, segment_ids], 
             outputs=[log_probs])

'\n# create our own layer to tune\noutput_weights = tf.Variable(\n  tf.random.truncated_normal([num_labels, hidden_size],stddev=0.02),\n  name="output_weights")\n\noutput_bias = tf.Variable(tf.zeros([num_labels]),\n                       name="output_bias")\n\n    #Dropout helps prevent overfitting\noutput_layer = tf.nn.dropout(output_layer, rate=0.1)\n\nlogits = tf.matmul(output_layer, output_weights, transpose_b=True)\nlogits = tf.nn.bias_add(logits, output_bias)\nlog_probs = tf.nn.log_softmax(logits, axis=-1)\n'

## Set up tokenizer
Initialize a FullTokenizer by loading a `vocab_file` from the bert model layer and `do_lower_case`.  for some reason, this step is missing the the hacked together examples (which I based this off of.)

In [13]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

# Prepare data
With these steps, we can generate BERT contextualized embedding vectors for our sentences!  Dont' forget to add [CLS] and [SEP] separator tokens to keep the original format!

In [14]:
#tokenizer = FullTokenizer
def convert_to_features(sentences):

    """s_list = []
    s_list.append("This is a lame sentence")
    s_list.append("This is a nice sentence.")
    """
    
    input_ids = []
    input_masks = []
    input_segments = []

    for s in sentences:
        #s = "This is a nice sentence."

        stokens = tokenizer.tokenize(s)
        stokens = stokens[:max_seq_length-2]
        stokens = ["[CLS]"] + stokens + ["[SEP]"]

        input_ids.append(get_ids(stokens, tokenizer, max_seq_length))
        input_masks.append(get_masks(stokens, max_seq_length))
        input_segments.append(get_segments(stokens, max_seq_length))

    return np.array(input_ids), np.array(input_masks), np.array(input_segments)
#pool_embs, all_embs = model.predict([[input_ids], [input_masks], [input_segments]])


In [15]:
input_word_ids, input_mask, segment_ids = convert_to_features(x_train)
test_input_word_ids, test_input_mask, test_segment_ids = convert_to_features(x_test)

# Convert labels into one-hot encoding
y_train_one_hot = tf.one_hot(y_train, depth=num_labels, dtype=tf.float32)
y_test_one_hot = tf.one_hot(y_test, depth=num_labels, dtype=tf.float32)


In [16]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [ ]:
model.fit([input_word_ids, input_mask, segment_ids], y_train_one_hot,
           epochs=1, batch_size=16, shuffle=True, verbose=1)

Train on 1024 samples
Epoch 1/3
1024/1024 - 252s - loss: 0.6931 - accuracy: 0.4941
Epoch 2/3


In [42]:
type(y_test)

numpy.ndarray

# Prediction
With these steps, we can generate BERT contextualized embedding vectors for our sentences!  Dont' forget to add [CLS] and [SEP] separator tokens to keep the original format!

In [ ]:
p_predicted_labels, p_log_probs = model.predict([input_ids, input_masks, input_segments])


In [ ]:
print(p_log_probs)